In [4]:
import pandas as pd
from pathlib import Path

raw_path = Path("../data/raw/online_retail_II.xlsx")

# Try first sheet; if it errors, we'll list sheets next
df = pd.read_excel(raw_path)

df.head(), df.shape

(  Invoice StockCode                          Description  Quantity  \
 0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
 1  489434    79323P                   PINK CHERRY LIGHTS        12   
 2  489434    79323W                  WHITE CHERRY LIGHTS        12   
 3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
 4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
 
           InvoiceDate  Price  Customer ID         Country  
 0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
 1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
 2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
 3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
 4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  ,
 (525461, 8))

In [5]:
print(df.columns)

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')


In [7]:
# 1) Normalize column names
df = df.rename(columns={
    "Customer ID": "CustomerID",
    "Price": "UnitPrice",
}).copy()

# 2) Parse dates
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# 3) Core cleaning rules
df = df.dropna(subset=["Invoice", "InvoiceDate", "StockCode", "Quantity", "UnitPrice"])

df["Invoice"] = df["Invoice"].astype(str).str.strip()
df["StockCode"] = df["StockCode"].astype(str).str.strip()

# Remove cancellations + returns
df = df.loc[~df["Invoice"].str.startswith("C")].copy()
df = df.loc[df["Quantity"] > 0].copy()

# Remove invalid prices
df = df.loc[df["UnitPrice"] > 0].copy()

#Revenue
df["LineRevenue"] = df["Quantity"] * df["UnitPrice"]

# 4) Weekly aggregation (week starts Monday)
df["WeekStart"] = df["InvoiceDate"].dt.to_period("W-MON").dt.start_time

weekly = (
    df.groupby(["WeekStart", "StockCode"], as_index=False)
        .agg(
            weekly_units=("Quantity", "sum"),
            weekly_revenue=("LineRevenue", "sum"),
            n_orders=("Invoice", "nunique"),
            n_customers=("CustomerID", "nunique")
        )
        .sort_values(["StockCode", "WeekStart"])
)

weekly.head(), weekly.shape

(      WeekStart StockCode  weekly_units  weekly_revenue  n_orders  n_customers
 0    2009-12-01     10002           143          121.55        11           11
 2477 2009-12-08     10002            59           51.85         6            4
 4873 2009-12-15     10002            12           10.20         1            1
 7087 2009-12-22     10002             2            2.55         2            1
 8263 2009-12-29     10002             3            2.55         1            1,
 (101186, 6))

In [8]:
out_path = Path("../data/processed/weekly_sku_demand.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)
weekly.to_csv(out_path, index=False)
out_path

WindowsPath('../data/processed/weekly_sku_demand.csv')

In [10]:
# Count how many weeks each SKU appears
sku_weeks = weekly.groupby("StockCode")["WeekStart"].nunique().sort_values(ascending=False)

# Recommended filter: keep SKUs with >= 20 weeks of sales history
keep_skus = sku_weeks[sku_weeks >= 20].index

weekly_f = weekly[weekly["StockCode"].isin(keep_skus)].copy()

weekly_f["StockCode"].nunique(), weekly_f.shape

(2222, (83612, 6))

In [11]:
import numpy as np

def backtest_one_sku(sku_df, horizon=4, min_train=20):
    # sku_df: WeekStart, weekly_units sorted
    y = sku_df["weekly_units"].to_numpy()
    weeks = sku_df["WeekStart"].to_numpy()

    rows = []
    # forecast 4 weeks ahead from multiple cutoffs
    for t in range(min_train, len(y) - horizon):
        train = y[:t]
        test = y[t:t+horizon]

        # Baselines
        naive = np.repeat(train[-1], horizon)
        ma4 = np.repeat(np.mean(train[-4:]) if len(train) >= 4 else np.mean(train), horizon)

        rows.append({
            "cutoff_week": weeks[t-1],
            "mae_naive": np.mean(np.abs(test - naive)),
            "mae_ma4": np.mean(np.abs(test - ma4)),
            "rmse_naive": np.sqrt(np.mean((test - naive)**2)),
            "rmse_ma4": np.sqrt(np.mean((test - ma4)**2)),
        })

    return pd.DataFrame(rows)

# Pick a representative SKU (most weeks of data)
top_sku = sku_weeks.index[0]
sku_df = weekly_f[weekly_f["StockCode"] == top_sku].sort_values("WeekStart")

bt = backtest_one_sku(sku_df, horizon=4, min_train=20)
bt.describe()

,mae_naive,mae_ma4,rmse_naive,rmse_ma4
count,30.000000,30.000000,30.000000,30.000000
mean,205.475000,173.729167,261.870045,215.756429
std,173.310046,85.686340,192.922545,122.023064
min,68.500000,55.500000,81.329576,69.932110
25%,87.375000,90.593750,113.263069,102.869634
50%,157.375000,164.375000,209.359110,195.425768
75%,239.937500,247.000000,356.098919,290.177785
max,950.000000,330.125000,952.322162,482.108714


In [12]:
top50 = sku_weeks[sku_weeks >= 20].head(50).index

all_bt = []
for sku in top50:
    sku_df = weekly_f[weekly_f["StockCode"] == sku].sort_values("WeekStart")
    res = backtest_one_sku(sku_df, horizon=4, min_train=20)
    if not res.empty:
        res["StockCode"] = sku
        all_bt.append(res)

all_bt = pd.concat(all_bt, ignore_index=True)

all_bt[["mae_naive","mae_ma4","rmse_naive","rmse_ma4"]].mean()

mae_naive     121.817167
mae_ma4       102.124833
rmse_naive    148.013970
rmse_ma4      127.047790
dtype: float64